# Back testing Code

In [1]:
import pyupbit
import numpy as np
import time
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from tqdm import tqdm

In [2]:
coin_type = "KRW-BTC"
# OHLCV: Open High Low Close Volume. 당일 시가, 고가, 저가, 종가, 거래량에 대한 data
df = pyupbit.get_ohlcv(coin_type, count=230) # 7 days

In [3]:
# ==========================================================
# 전략: 변동성 돌파


# 변동폭*k 계산. (고가-저가)*k value
k = 0.032
df['range'] = (df['high'] - df['low']) * k

# target(매수가), range column을 한 칸씩 밑으로 내림
# 어제 가격을 오늘 반영해서 써야 하기 때문
df['target'] = df['open'] + df['range'].shift(1) 

# ==========================================================

In [4]:
# np.where(조건문, 참일때 값, 거짓일 때 값)
fee = 0.0005

# 수익률 = 목표가/종가 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - fee, 1)
df['ror']

2021-08-27 09:00:00    1.000000
2021-08-28 09:00:00    0.998326
2021-08-29 09:00:00    0.996128
2021-08-30 09:00:00    0.972982
2021-08-31 09:00:00    0.988194
                         ...   
2022-04-09 09:00:00    1.004465
2022-04-10 09:00:00    0.990211
2022-04-11 09:00:00    0.946581
2022-04-12 09:00:00    1.008063
2022-04-13 09:00:00    1.000000
Name: ror, Length: 230, dtype: float64

In [5]:
# 누적 곱 계산(cumprod) => 누적 수익률
df['hpr'] = df['ror'].cumprod()
df['hpr']

2021-08-27 09:00:00    1.000000
2021-08-28 09:00:00    0.998326
2021-08-29 09:00:00    0.994460
2021-08-30 09:00:00    0.967592
2021-08-31 09:00:00    0.956169
                         ...   
2022-04-09 09:00:00    0.855723
2022-04-10 09:00:00    0.847347
2022-04-11 09:00:00    0.802082
2022-04-12 09:00:00    0.808549
2022-04-13 09:00:00    0.808549
Name: hpr, Length: 230, dtype: float64

In [6]:
# Draw Down 계산. (누적 최대 값과 현재 hpr 차이/ 누적 최대값*100)
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() *100
df

,open,high,low,close,volume,value,range,target,ror,hpr,dd
2021-08-27 09:00:00,55255000.0,57600000.0,54616000.0,57401000.0,8324.331968,4.660757e+11,95488.0,NaN,1.000000,1.000000,0.000000
2021-08-28 09:00:00,57400000.0,57885000.0,56704000.0,57428000.0,5552.262749,3.186951e+11,37792.0,57495488.0,0.998326,0.998326,0.167380
2021-08-29 09:00:00,57428000.0,58000000.0,56500000.0,57272000.0,11623.618673,6.647657e+11,48000.0,57465792.0,0.996128,0.994460,0.553962
2021-08-30 09:00:00,57272000.0,57479000.0,55555000.0,55800000.0,14613.333391,8.275263e+11,61568.0,57320000.0,0.972982,0.967592,3.240774
2021-08-31 09:00:00,55800000.0,56333000.0,55022000.0,55230000.0,11075.546983,6.165163e+11,41952.0,55861568.0,0.988194,0.956169,4.383109
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-09 09:00:00,52934000.0,53355000.0,52754000.0,53258000.0,2253.816753,1.195229e+11,19232.0,52994864.0,1.004465,0.855723,31.840946
2022-04-10 09:00:00,53295000.0,53950000.0,52462000.0,52819000.0,2289.320776,1.217211e+11,47616.0,53314232.0,0.990211,0.847347,32.508150
2022-04-11 09:00:00,52799000.0,53060000.0,50010000.0,50050000.0,5382.926050,2.779839e+11,97600.0,52846616.0,0.946581,0.802082,36.113530
2022-04-12 09:00:00,50050000.0,51195000.0,49608000.0,50577000.0,4730.164656,2.381793e+11,50784.0,50147600.0,1.008063,0.808549,35.598431


In [7]:
print(f"MDD(%): {df['dd'].max()}") 

MDD(%): 43.5592132976547


In [8]:
df['hpr'][-1]

0.8085489060505305

# Back testing function

In [19]:
def back_testing(coin_type, k_value, term, prt:bool=False):
    time.sleep(0.1) # Redundant
    try:
        df = pyupbit.get_ohlcv(coin_type, count=term) # Redundant
        df['range'] = (df['high'] - df['low']) * k_value
        df['target'] = df['open'] + df['range'].shift(1) 

        fee = 0.0032
        df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - fee, 1)

        df['hpr'] = df['ror'].cumprod()
        df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100

        pct = df['hpr'][-1]
        profit = round(pct-1, 5) * 100

        if (prt):
            print_str = f"{coin_type} {term}일간 수익률: {profit}%, MDD: {df['dd'].max()}"
            print(print_str)

    except Exception as e:
        print(e)
        time.sleep(0.1)
        return 7210, 7210
    
    return profit, df['dd'].max()

In [10]:
back_testing("KRW-ETH", 0.0032, 60, True)

KRW-ETH 60일간 수익률: -5.888%, MDD: 21.042627394966953


(-5.888, 21.042627394966953)

In [11]:
back_testing("KRW-ETH", 0.451, 60, True)

KRW-ETH 60일간 수익률: -16.115%, MDD: 20.278313895826894


(-16.115, 20.278313895826894)

In [12]:
back_testing("KRW-BTC", 0.0032, 60, True)

KRW-BTC 60일간 수익률: -14.75%, MDD: 18.62600516062338


(-14.75, 18.62600516062338)

In [13]:
back_testing("KRW-BTC", 0.451, 60, True)

KRW-BTC 60일간 수익률: 5.373%, MDD: 9.090997560744995


(5.373, 9.090997560744995)

# Finding Hyper parameter

In [14]:
def find_hyper_k(coin_type, term):
    # k value의 최적값을 찾기 위해 backtesting하며 수익률을 확인한다.
    df = pd.DataFrame([[0,0,0]], columns=['수익률', 'MDD%', 'k-value'])

    for i in tqdm(np.arange(0, 0.5, 0.001), desc='Progress', mininterval=0.1):
        profit, mdd = back_testing(coin_type, i, term, False)

        max_profit = df['수익률'].max()
        if profit >= max_profit:
            df = df.append(pd.Series([profit, mdd, i], index=df.columns), ignore_index=True)

    filter = df['수익률'] == df['수익률'].max()
    hyper_k = df[filter].iloc[0,2]
    
    return hyper_k:

In [15]:
filter = df['수익률'] == df['수익률'].max()
df[filter]

KeyError: '수익률'

In [ ]:
filt2 = df['수익률'] == 7210
df[filt2]

In [ ]:
df

In [22]:
find_hyper_k("KRW-NEAR", 11)

Progress: 100%|██████████| 500/500 [01:34<00:00,  5.28it/s]


0.048

In [ ]:
df

In [20]:
# k value의 최적값을 찾기 위해 backtesting하며 수익률을 확인한다.
df = pd.DataFrame([[0,0,0]], columns=['수익률', 'MDD%', 'k-value'])

for i in tqdm(np.arange(0, 0.5, 0.001), desc='Progress', mininterval=0.1):
    profit, mdd = back_testing("KRW-KNC", i, 14, False)

    max_profit = df['수익률'].max()
    if profit >= max_profit:
        df = df.append(pd.Series([profit, mdd, i], index=df.columns), ignore_index=True)
filter = df['수익률'] == df['수익률'].max()
hyper_k = df[filter].iloc[0,2]

Progress: 100%|██████████| 500/500 [01:32<00:00,  5.38it/s]


In [21]:
filter = df['수익률'] == df['수익률'].max()
df[filter]

,수익률,MDD%,k-value
1,15.04,9.019347,0.0
